In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv(r"listings.csv")
df = df[['price', 'host_is_superhost','host_listings_count',
                      'accommodates', 'bathrooms_text', 'beds', 'minimum_nights', 'maximum_nights', 
                      'number_of_reviews', 'review_scores_rating', 'property_type']]
df.columns

Index(['price', 'host_is_superhost', 'host_listings_count', 'accommodates',
       'bathrooms_text', 'beds', 'minimum_nights', 'maximum_nights',
       'number_of_reviews', 'review_scores_rating', 'property_type'],
      dtype='object')

In [27]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

listings = train_set.copy()
# does not include longitude and latitude

In [81]:
def extract_price(price):
    #converts price column to floats - DOES NOT CURRENTLY WORK
    
    if isinstance(price, str):
        price = float(price.replace('$','').replace(',',''))
    return price

#listings['price'] = clean_price(listings['price'])

def clean_bathrooms(bathrooms_text):
    #cleans the bathrooms... har har har... no actually it cleans the bathrooms_text column - DOES NOT CURRENTLY WORK
    if isinstance(property_type_col, pd.DataFrame):
        property_type_col = property_type_col.squeeze() 
    bathrooms_text = bathrooms_text.copy()
    pattern = r'(\d.?\d?)\s'
    pattern2 = r'(Half)'
    bathrooms_text.loc[bathrooms_text.str.contains(pattern2, na=False)] = .5
    bathrooms_text = bathrooms_text.str.extract(pattern)
    return pd.DataFrame(bathrooms_text.astype(float), columns=['bathrooms_text'])

#listings['bathrooms'] = clean_bathrooms(listings['bathrooms_text'])
#listings.drop(columns='bathrooms_text', inplace=True)


def clean_property_type(property_type_col):
    #consolidate the property_type var to a few common categories - THIS VERSION DOES NOT WORK
    if isinstance(property_type_col, pd.DataFrame):
        property_type_col = property_type_col.squeeze() 
     
    property_type_col = property_type_col.copy()
    property_type_col.loc[property_type_col.str.contains(r'Entire')] = 'Entire Unit'
    property_type_col.loc[property_type_col.str.contains(r'Tiny home')] = 'Entire Unit'
    property_type_col.loc[property_type_col.str.contains(r'[Rr]oom')] = 'Single Room'
    property_type_col.loc[property_type_col.str.contains(r'Camp')] = 'Camping'

    property_type_col.loc[~property_type_col.isin(['Camping', 'Single Room', 'Entire Unit'])] = np.nan
    return pd.DataFrame(property_type_col, columns=['property_type'])

#listings['property_type'] = clean_property_type(listings['property_type'])

In [85]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

num_pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

cat_pipe = make_pipeline(   # host_is_superhost
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(drop='if_binary')
)

bathrooms_pipeline = make_pipeline(
    FunctionTransformer(func=clean_bathrooms),
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

property_type_pipeline = make_pipeline(
    FunctionTransformer(func=clean_property_type),
    SimpleImputer(strategy='most_frequent')
)

clean_price_pipeline = make_pipeline(
    FunctionTransformer(func=extract_price),
    SimpleImputer(strategy='mean'),
    FunctionTransformer(np.log),
    StandardScaler()
)

preprocessing = ColumnTransformer([
    ('clean_price', clean_price_pipeline, ['price']),
    ('clean_bathrooms', bathrooms_pipeline, ['bathrooms_text']),
    ('clean_property_type', property_type_pipeline, ['property_type']),
    ('cat', cat_pipe, make_column_selector(dtype_include=object)),
],
remainder=num_pipe)

In [107]:
# THIS FUNCTION WORKS IN A PIPELINE

def clean_property_type(df):
    col_name = 'property_type'
    df = df.copy()
    df[col_name] = df[col_name].str.replace(r'Entire|Tiny home', 'Entire Unit', regex=True)
    df[col_name] = df[col_name].str.replace(r'[Rr]oom', 'Single Room', regex=True)
    df[col_name] = df[col_name].str.replace(r'Camp', 'Camping', regex=True)
    df[col_name] = df[col_name].where(df[col_name].isin(['Camping', 'Single Room', 'Entire Unit']), np.nan)
    return df

property_type_pipeline = make_pipeline(
    FunctionTransformer(func=clean_property_type),
    SimpleImputer(strategy='most_frequent')
)

t = df[['property_type']].copy()

property_type_pipeline.fit_transform(t)

array([['Entire Unit'],
       ['Entire Unit'],
       ['Entire Unit'],
       ...,
       ['Entire Unit'],
       ['Entire Unit'],
       ['Entire Unit']], dtype=object)

In [108]:
# DOES NOT WORK - different attempt at property type cleaning

def consolidate_categories(category):
    if 'Entire' in category:
        return 'Entire'
    elif 'Private room' in category:
        return 'Private room'
    elif 'Room in' in category:
        return 'Shared room'
    else:
        return 'Other'
    
t = df[['property_type']].copy()
transformer = FunctionTransformer(func=lambda x: x.apply(consolidate_categories))
transformer.transform(t)

property_type    Other
dtype: object

In [104]:
t = df[['price']].copy()

# Define the extract_price function
def extract_price(price_series):
    # Converts price column to floats
    return price_series.apply(lambda x: float(x.replace('$','').replace(',','')) if isinstance(x, str) else x)

# Create a FunctionTransformer
clean_price = FunctionTransformer(func=extract_price)

# Define the pipeline
clean_price_pipeline = make_pipeline(
    clean_price,
    SimpleImputer(strategy='mean'),
    FunctionTransformer(np.log1p),  # Use np.log1p to avoid log(0) issues
    StandardScaler()
)

clean_price_pipeline.fit_transform(t)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '$100.00'

*0. remove outliers*
1. Response variable(price): (clean_price function applied to price column, log transformation applied to price column.
2. clean_bathrooms applied to the bathrooms variable
3. clean_property_type to property_type column
4. numeric columns imputed and scaled (num_pipeline)
5. cat columns imputed (most frequent) and one hot encoded (cat_pipeline)

In [ ]:
from sklearn.ensemble import IsolationForest

rem_outliers = IsolationForest()

outliers_array = rem_outliers.fit_predict(data)
data = data.iloc[outliers_array == 1]

def remove_outliers(data, outlier_array):
    return data.iloc[outlier_array == 1]